In [1]:
import argparse
import os
from datetime import date
from datetime import datetime
import datetime as dt
import pandas as pd
import numpy as np
from numpy.random import randint


In [2]:
D = pd.read_csv("Day4_Trades_SPY.csv")

In [3]:
D.head(10)

,Time,PRICE,SIZE,EXCHANGE,PARTICIPANT_TIME
0,09:30:00.002130,236.28,100,Z,09:30:00.000093
1,09:30:00.002130,236.29,100,K,09:30:00.000101
2,09:30:00.002130,236.11,1,K,09:30:00.001527
3,09:30:00.002141,236.17,20,K,09:30:00.001640
4,09:30:00.002180,236.03,10,K,09:30:00.001714
5,09:30:00.002218,236.46,8,K,09:30:00.001797
6,09:30:00.002328,236.05,25,K,09:30:00.001927
7,09:30:00.002657,236.04,7,K,09:30:00.002229
8,09:30:00.002828,236.48,100,Z,09:30:00.002380
9,09:30:00.002852,236.30,100,K,09:30:00.002382


In [4]:
D.describe()

,PRICE,SIZE
count,2.803902e+06,2.803902e+06
mean,2.368660e+02,9.693719e+01
std,4.192842e+00,1.385586e+03
min,2.278100e+02,1.000000e+00
25%,2.332400e+02,1.400000e+01
50%,2.374100e+02,8.500000e+01
75%,2.400500e+02,1.000000e+02
max,2.484900e+02,2.094942e+06


In [5]:
def computeMinute(s):
    d = pd.to_datetime(s)
    h = d.hour 
    m = d.minute
    return 60*h + m

def computeSecond(s):
    d = pd.to_datetime(s)
    h = d.hour
    m = d.minute
    sec = d.second
    return 60*(60*h+m)+sec

def addMinutesColumn(data):
    data['Minute'] = data.apply(lambda x: computeMinute(x['Time']),axis=1)
    return data

def addSecondsColumn(data):
    data['Second'] = data.apply(lambda x: computeSecond(x['Time']),axis=1)
    return data

In [6]:
D = addMinutesColumn(D)

In [ ]:
D = addSecondsColumn(D)

In [ ]:
D.head()

In [ ]:
D['Minute'].value_counts()

In [ ]:
D.describe()

In [17]:
D.groupby(['Minute'])['PRICE'].first()

Minute
570    236.28
571    238.10
572    239.93
573    239.58
574    241.67
        ...  
956    240.35
957    239.75
958    237.19
959    238.07
960    240.86
Name: PRICE, Length: 377, dtype: float64

In [18]:
#lets compute a few stats per minute and fill in the gaps for missing minutes. For example:
D.groupby(['Minute'])['SIZE'].sum()

Minute
570    2415972
571    1620751
572    1477235
573    1387699
574    1051070
        ...   
956    3291039
957    3435449
958    3165541
959    4353150
960    2107160
Name: SIZE, Length: 377, dtype: int64

In [19]:
#now let's do a few of these and join them together by minute:
Volume_by_minute=pd.DataFrame(D.groupby(['Minute'])['SIZE'].sum()).rename({'SIZE':'VOLUME'},axis=1)
Num_trades_by_minute=pd.DataFrame(D.groupby(['Minute'])['PRICE'].count()).rename({'PRICE':'NUM_TRADES'},axis=1)

max_price_by_minute=pd.DataFrame(D.groupby(['Minute'])['PRICE'].max()).rename({'PRICE':'MAX_PRICE'},axis=1)
min_price_by_minute=pd.DataFrame(D.groupby(['Minute'])['PRICE'].min()).rename({'PRICE':'MIN_PRICE'},axis=1)

M = Volume_by_minute.join(Num_trades_by_minute)
M= M.join(max_price_by_minute)
M = M.join(min_price_by_minute)



In [20]:
M

,VOLUME,NUM_TRADES,MAX_PRICE,MIN_PRICE
Minute,,,,
570,2415972,12993,239.23,236.01
571,1620751,12169,240.68,236.03
572,1477235,11760,240.52,238.67
573,1387699,11952,243.54,239.44
574,1051070,11460,242.36,239.99
...,...,...,...,...
956,3291039,26422,240.81,239.23
957,3435449,22225,240.18,236.32
958,3165541,19772,238.34,236.25


In [21]:
#now let's fill in the missing minutes
M = M.reindex(list(range(M.index.min(),M.index.max()+1)),fill_value=0)

In [22]:
M

,VOLUME,NUM_TRADES,MAX_PRICE,MIN_PRICE
Minute,,,,
570,2415972,12993,239.23,236.01
571,1620751,12169,240.68,236.03
572,1477235,11760,240.52,238.67
573,1387699,11952,243.54,239.44
574,1051070,11460,242.36,239.99
...,...,...,...,...
956,3291039,26422,240.81,239.23
957,3435449,22225,240.18,236.32
958,3165541,19772,238.34,236.25


In [23]:
M[M['VOLUME']==0]

,VOLUME,NUM_TRADES,MAX_PRICE,MIN_PRICE
Minute,,,,
777,0,0,0.0,0.0
778,0,0,0.0,0.0
779,0,0,0.0,0.0
780,0,0,0.0,0.0
781,0,0,0.0,0.0
782,0,0,0.0,0.0
783,0,0,0.0,0.0
784,0,0,0.0,0.0
785,0,0,0.0,0.0


In [24]:
#let's look at the minutes right before the halt:
M.loc[760:777]

,VOLUME,NUM_TRADES,MAX_PRICE,MIN_PRICE
Minute,,,,
760,538704,6239,238.31,236.88
761,695713,7168,237.91,236.54
762,658357,5970,237.72,236.61
763,274989,3010,237.61,236.74
764,504686,5930,238.68,236.96
765,513149,5674,238.76,237.32
766,408274,4287,238.52,237.02
767,476818,4116,237.56,236.65
768,643334,6907,237.73,236.27


In [25]:
M['price_gap'] = M.apply(lambda x: x['MAX_PRICE']-x['MIN_PRICE'],axis=1)

In [26]:
M.loc[760:777]

,VOLUME,NUM_TRADES,MAX_PRICE,MIN_PRICE,price_gap
Minute,,,,,
760,538704,6239,238.31,236.88,1.43
761,695713,7168,237.91,236.54,1.37
762,658357,5970,237.72,236.61,1.11
763,274989,3010,237.61,236.74,0.87
764,504686,5930,238.68,236.96,1.72
765,513149,5674,238.76,237.32,1.44
766,408274,4287,238.52,237.02,1.50
767,476818,4116,237.56,236.65,0.91
768,643334,6907,237.73,236.27,1.46


In [ ]:
#proposed model: if price gap > $1.50 and number of trades > 8000 in a single minute, 
#guess that a circuit breaker is coming? 

In [17]:
def modelTrigger(num_trades,price_gap):
    if(price_gap > 1.5):
        if(num_trades > 8000):
            return 1
    return 0

In [18]:
M['Trigger'] = M.apply(lambda x: modelTrigger(x['NUM_TRADES'],x['price_gap']),axis=1)

In [19]:
M[M['Trigger']==1]

,VOLUME,NUM_TRADES,MAX_PRICE,MIN_PRICE,price_gap,Trigger
Minute,,,,,,
570,2415972,12993,239.23,236.01,3.22,1
571,1620751,12169,240.68,236.03,4.65,1
572,1477235,11760,240.52,238.67,1.85,1
573,1387699,11952,243.54,239.44,4.10,1
574,1051070,11460,242.36,239.99,2.37,1
...,...,...,...,...,...,...
955,3062583,21747,240.52,238.55,1.97,1
956,3291039,26422,240.81,239.23,1.58,1
957,3435449,22225,240.18,236.32,3.86,1


In [24]:
M.loc[775]

VOLUME        811702.00
NUM_TRADES      8075.00
MAX_PRICE        237.07
MIN_PRICE        235.55
price_gap          1.52
Trigger            1.00
Name: 775, dtype: float64

In [20]:
M.loc[750:780]

,VOLUME,NUM_TRADES,MAX_PRICE,MIN_PRICE,price_gap,Trigger
Minute,,,,,,
750,460084,4637,239.62,238.16,1.46,0
751,330355,3250,238.90,238.02,0.88,0
752,483583,5222,238.72,237.65,1.07,0
753,451898,4347,238.82,237.72,1.10,0
754,485094,4815,238.74,237.87,0.87,0
755,640644,6598,238.69,236.96,1.73,0
756,521923,5163,238.10,236.85,1.25,0
757,438082,4678,238.09,236.80,1.29,0
758,672854,6248,238.01,236.79,1.22,0
